In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
print(data.shape)
data.head(2)

(2396804, 12)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [11]:
result.loc[[0],['actual']].values

array([[array([  821867,   834484,   856942,   865456,   889248,   907957,
                 914190,   943316,   951954,   954486,   958046,   962568,
                 969231,   971585,   979707,   986947,   990656,   995242,
                1004906,  1005186,  1042083,  1050310,  1060819,  1062002,
                1064441,  1069733,  1074612,  1082185,  1131115,  1132771,
                6534544, 13876341, 15971874, 17178953,   883616,   917704,
                 931860,   961554,  1002032,  1031190,  8090541,  8293439,
                9297615,  9527329, 15926712,  1049998,   861272,   869465,
                 877373,   908213,   933913,   940947,   945809,   959316,
                 978974,  1031697,  1041796,  1048918,  1081189,  1101422,
                1115576,  1122428,  1132231,  1132814,  5577022,  8091601,
                9296986,  9677939, 10356149, 13417048, 15741823, 15830875],
              dtype=int64)                                                 ]],
      dtype=object)

In [5]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    return recs.tolist()

In [6]:
items = data_train.item_id.unique().tolist() #список всех товаров из обучающей выборки
len(items)

86865

In [7]:
%%time
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 21.1 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[777800, 911886, 2047921, 5587520, 1766585]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8019544, 847218, 1011300, 42319, 7168885]"


In [8]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    return recs.tolist()

In [9]:
%%time
#popularr = data_train.groupby('item_id')['sales_value'].sum().reset_index()
#popularr.sort_values('sales_value', ascending=False, inplace=True)
#popularr.head()
popular_recs = popularity_recommendation(data_train, n=5)
# Можно так делать, так как рекомендация не зависит от юзера
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 600 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[777800, 911886, 2047921, 5587520, 1766585]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8019544, 847218, 1011300, 42319, 7168885]","[6534178, 6533889, 1029743, 6534166, 1082185]"


Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу
Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [10]:
weight_item = data.groupby('item_id')['quantity'].sum().reset_index()
print(weight_item.shape)
weight_item.head()

(89051, 2)


,item_id,quantity
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [11]:
%%time
sum_weight=0
for kl in range(weight_item.shape[0]):
    sum_weight+=weight_item.loc[[kl],['quantity']].values[0][0]
print(sum_weight)

240582404
Wall time: 1min 45s


In [12]:
#weight_item['quat1'] = weight_item[''].apply(lambda x: random_recommendation(items, n=5))
weight_item['quant1']=weight_item['quantity'].div(sum_weight) 
#weight_item.head()
weight_item=weight_item.drop('quantity', axis=1)
weight_item.head(3)

,item_id,quant1
0,25671,2.493948e-08
1,26081,4.156580e-09
2,26093,4.156580e-09


In [13]:
def weighted_random_recom(items_weights, n=5):
    """Случайные рекоммендации
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = weight_item['item_id'].tolist()
    p = weight_item['quant1'].tolist()
    recs = np.random.choice(items, size=n, replace=False,p=p)
    return recs.tolist()

In [14]:
%%time
result['random_recom_weight'] = result['user_id'].apply(lambda x: weighted_random_recom(weight_item, n=5))
result.head(2)

Wall time: 46.4 s


,user_id,actual,random_recommendation,popular_recommendation,random_recom_weight
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[777800, 911886, 2047921, 5587520, 1766585]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 6533889, 1404121, 862349, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8019544, 847218, 1011300, 42319, 7168885]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 397896, 1426702, 6533889, 6534166]"


Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [15]:
def precision_at_k_my(recommended_list, bought_list,k=5):
    #nnn=bought_list.strip('[]')
    #new_rec=nnn.split(',')
    #int_lst = [int(x) for x in new_rec]
    #bought_lst = np.array(int_lst)
    bought_list = np.array(bought_list)  #[5586238, 1015228, 866118, 2416733, 2603573]
    #print(bought_list.dtype)
    #print(int_lst)
    recommended_list = np.array(recommended_list[:k])
    #print(recommended_list)
    #recommended_list=recommended_list[:k]
    #nnn=recommended_list.strip('[]')
    #new_rec=nnn.split()
    #int_lst = [int(x) for x in new_rec]
    #int_lst = np.array(int_lst)
    #print(int_lst)
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    return flags.sum() / len(recommended_list)    

In [16]:
result.apply(lambda row: precision_at_k_my(row['actual'], row['random_recommendation']), axis=1).mean()

0.0

In [17]:
result.apply(lambda row: precision_at_k_my(row['actual'], row['popular_recommendation']), axis=1).mean()

0.027138426379366695

In [18]:
result.apply(lambda row: precision_at_k_my(row['actual'], row['random_recom_weight']), axis=1).mean()

0.018307215148547173

Лучший результат для функции popular_recommendation, хуже для weighted_random_recom. Для random_recommendation совпадений нет.

In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [7]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()#id самых популярных товаров
top_5000[:6] #получили индексыс 5000 самых популярных товаров

[6534178, 6533889, 6534166, 6544236, 1404121, 397896]

In [17]:
data_train['item_id'].nunique()

86865

In [8]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\Roman\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
data_train['item_id'].nunique()

5001

In [10]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

In [11]:
user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
print(userids)
print(itemids)

[   1    2    3 ... 2498 2499 2500]
[  202291   397896   420647 ... 15927661 15927850 16809471]


In [13]:
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

In [14]:
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [25]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/2499 [00:00<?, ?it/s]

In [ ]:
'''ломается ядро!!!
%%time
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True) #'''

In [15]:
%%time
model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/2499 [00:00<?, ?it/s]

Wall time: 1.01 s


In [ ]:
'''ломается ядро!!!
recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False) '''

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [21]:
# your_code Т.к. расчет из моделей не получился, читаем рекомендации из predictions_basic.csv и считаем precision
result2 = pd.read_csv('predictions_basic.csv')
print(result2.shape)
result2.head(2)

(2042, 8)


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [82]:
#изменим функцию
def precision_at_k_my2(recommended_list, bought_list,k=5):
    #bought_list = np.array(bought_list)  #[5586238, 1015228, 866118, 2416733, 2603573]
    #print(bought_list) #.dtype
    nnn=bought_list.strip('[]')
    new_rec=nnn.split(',')
    int_lst = [int(x) for x in new_rec]
    bought_list1 = np.array(int_lst)
    #print(bought_lst1)
    
    #print(recommended_list)
    nnn=recommended_list.strip('[]')
    new_rec=nnn.split()
    int_lst = [int(x) for x in new_rec]
    recommended_list1 = np.array(int_lst[:k])
    #print(int_lst)
    '''
    print(int_lst)
    recommended_list = np.array(recommended_list[:k])
    
    #recommended_list=recommended_list[:k]
    '''
    flags = np.isin(bought_list1, recommended_list1)
    #print(flags)
    return flags.sum() / len(recommended_list1)    

In [83]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['random_recommendation']), axis=1).mean()

0.0

In [84]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['popular_recommendation']), axis=1).mean()

0.027138426379366695

In [85]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['itemitem']), axis=1).mean()

0.007476330395037537

In [86]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['cosine']), axis=1).mean()

0.006994776363042763

In [87]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['tfidf']), axis=1).mean()

0.007484492327783212

In [88]:
result2.apply(lambda row: precision_at_k_my2(row['actual'], row['own_purchases']), axis=1).mean()

0.027113940581129693

Лучший результат остался за popular_recommendation, чуть меньше own_purchases, остальные функции показывают худший результат (кроме random_recommendation, он нулевой) )  

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [89]:
# your_code
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
#popularity.head()
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()#id самых популярных товаров
top_5000[:6] #получили индексыс 5000 самых популярных товаров из набора data_train

[6534178, 6533889, 6534166, 6544236, 1404121, 397896]

In [90]:
len(top_5000)

5000

In [91]:
#data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
#df_5000=data_train[data_train['item_id'].isin(top_5000)]

In [92]:
#df_5000['item_id'].nunique()

5000

In [99]:
%%time
#popular_recs = popularity_recommendation(top_5000, n=5)
result2['popular_recom_top5000'] = result2['user_id'].apply(lambda x: top_5000[:5])
result.head(2)

Wall time: 73 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,popular_recom_top5000
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[6534178, 6533889, 6534166, 6544236, 1404121]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[6534178, 6533889, 6534166, 6544236, 1404121]"


In [112]:
def precision_at_k_my3(recommended_list, bought_list,k=5):
    bought_list = np.array(bought_list)  #[5586238, 1015228, 866118, 2416733, 2603573]
    nnn=recommended_list.strip('[]')
    new_rec=nnn.split()
    int_lst = [int(x) for x in new_rec]
    recommended_list1 = np.array(int_lst[:k])
    #print(int_lst)
    
    flags = np.isin(bought_list, recommended_list1)
    #print(flags)
    return flags.sum() / len(recommended_list1)

In [113]:
result2.apply(lambda row: precision_at_k_my3(row['actual'], row['popular_recom_top5000']), axis=1).mean()

0.01860104472739144

посчитанный popular_recommendation на top5000 по признаку quantity хуже, чем исходный popular_recommendation, но лучше, чем остальные расчеты.
Улучшить ItemItemRecommender не получилось из-за поломок ядра python при расчетах.